# Theory DB Scan

## Articles
https://www.sciencedirect.com/science/article/pii/S0957417417307698
https://www.sciencedirect.com/science/article/pii/S2352146518301820
https://www.sciencedirect.com/science/article/pii/S1877050915008741
https://github.com/turi-code/userguide/blob/master/clustering/dbscan.md

## Summary theory
4 method for clustering:
    1. partitioning approaches (where the number of clusters is pre-assigned)
    2. grid-based (where the object space is divided into a pre-assigned number of cells)
    3. hierarchical (where the data is organized in multiple levels) 
    4. density-based (where density notion is considered).

DBScan is density-based. It needs two parameters:
    1. Radius of a circle around the data point
    2. Minimum number of points that should be in the circle
    
DBSCAN-TE is used to group GPS points into clusters (stopping points) and
noise (moving points) due to their difference in spatial density, temporal 
sequence, and entropy index. Activity stops are then refined from stopping 
points by SVMs. The structure of the two-step methodology is shown in Fig. 1.

## Install packages
    1. link voor download packages https://www.lfd.uci.edu/~gohlke/pythonlibs/
    2. installeer: geopandas, GDAL, Fiona, Basemap, Shapely, Pyproj, wheel 
    3. Bestand van download naar script map anaconda 
    4. pip install bestand.whl (behalve bij geopandas, deze moet je installeren via GitHub)

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing
# import plotly.plotly as py 
# import plotly
# plotly.tools.set_credentials_file(username='candy_93j', api_key='pdU9f5lyr1CoSmwpj2r6')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
# from bokeh.plotting import figure, show, output_file
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, curdoc, show
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import VBar
output_notebook()

In [ ]:
#inladen csv (uitkomst vorige script)
df = pd.read_csv('XYT_speed_distance.csv')
df = df[:-1] #Laatste regel is NAN
df_old = df #Nodig voor merge verderop in script

In [ ]:
#Schaal lat, lon en speed voor dbscan
coords_speed = np.asarray(df[['Lat_a', 'Lon_a', "speed_kmu"]])
coords = preprocessing.scale(coords_speed)
# coords = np.asarray(df[['Lat_a', 'Lon_a']])

In [ ]:
##bereken EPS door K-distance elbow (Nu niet van toepassing)
# def k_distances2(x, k):
#     dim0 = x.shape[0]
#     dim1 = x.shape[1]
#     p=-2*x.dot(x.T)+np.sum(x**2, axis=1).T+ np.repeat(np.sum(x**2, axis=1),dim0,axis=0).reshape(dim0,dim0)
#     p = np.sqrt(p)
#     p.sort(axis=1)
#     p=p[:,:k]
#     pm= p.flatten()
#     pm= np.sort(pm)
#     return p, pm
# m, m2= k_distances2(coords, 5)
# plt.plot(m2)
# plt.ylabel("k-distances")
# plt.grid(True)
# plt.show()

In [ ]:
##Berekenen de clusters met DBSCAN
# # kms_per_radian = 6371.0088
# m_per_radian = 6371008.8
# # epsilon = 0.005 / kms_per_radian
# epsilon_m = 0.005 / m_per_radian #5 meter kwam uit k_distance
epsilon = 0.154152
min_samples = 3
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree').fit(coords)
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
print('Number of clusters: {}'.format(num_clusters))

#Clusterlabels naar column
df["cluster"] = cluster_labels

In [ ]:
##visualize clusters (Nu niet relevant)
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# n_clusters_1 = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

# unique_labels = set(cluster_labels)
# colors = [plt.cm.Spectral(each)
#           for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]

#     class_member_mask = (cluster_labels == k)

#     xy = coords[class_member_mask & core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=14)

#     xy = coords[class_member_mask & ~core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=6)

# plt.title('Estimated number of clusters: %d' % n_clusters_1)
# plt.show()

In [ ]:
#merge met de oude gegevens 
df = df[["VgNr", "cluster"]]
df_new = pd.merge(df_old, df, on="VgNr", how="left")

df_new.to_csv("total_eps015_MinSamples3.csv")

In [ ]:
#Clusters naar middelste XY
coords_new = np.asarray(df_new[['Lat_a', 'Lon_a', "speed_kmu"]])
clusters = pd.Series([coords_new[cluster_labels == n] for n in range(num_clusters)])
clusters = clusters[:-1]

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)

lats, lons, speed = zip(*centermost_points)
rep_points = pd.DataFrame({'Lon_a':lons, 'Lat_a':lats, 'speed_kmu':speed})

In [ ]:
##create cluster table 
kmu_per_cluster = df_new.groupby('cluster_x')['speed_kmu'].mean().reset_index(name='kmu_per_cluster')
NoP_per_cluster = df_new.groupby('cluster_x')["cluster_x"].count().reset_index(name='NoP_per_cluster')
NoP_kmu_per_cluster = pd.merge(kmu_per_cluster, NoP_per_cluster, on=["cluster_x"], how="left")
ID_centerpoint = df_new[["VgNr", "Lat_a", "Lon_a", "cluster_x"]]
ID_centerpoint = pd.merge(rep_points, ID_centerpoint, on=["Lon_a", "Lat_a"], how="left")
cluster_table = pd.merge(ID_centerpoint, NoP_kmu_per_cluster, on=["cluster_x"], how="left")
ct = cluster_table[cluster_table["kmu_per_cluster"] <=10]
ct.head(100)

ct.to_csv("ClusterTable_eps015_MinSamples_3.csv")

In [ ]:
# #Visualisatie 1: clusters lan - lon

# output to static HTML file
output_file("line.html")

p = figure(plot_width=500, plot_height=500)

# add a circle renderer with a size, color, and alpha
p.circle(x=ct["Lat_a"], y=ct["Lon_a"], size=20, color="navy", alpha=0.5)

# show the results
show(p)
# import plotly.offline as offline

# # Create a trace
# trace = go.Scatter(
#     x = ct["Lon_a"],
#     y = ct["Lat_a"],
#     mode = "markers"
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# iplot(data)


# # x = ct["Lon_a"]
# # y = ct["Lat_a"]
# # # offline.iplot([go.Histogram2dContour(x=x, y=y, contours=dict(coloring='heatmap')),
# #        go.Scatter(x=x, y=y, mode='markers', marker=dict(color='white', size=3, opacity=0.3))], show_link=False)

In [ ]:
# Visualisatie 2: aantal punten per cluster
x = ct["cluster_x"]
y = ct["NoP_per_cluster"]

plot = figure(plot_width=500, plot_height=500)
source = ColumnDataSource(dict(x=x,top=y,))

glyph = VBar(x="x", top="top", bottom=0, width=0.5, fill_color="#b3de69")
plot.add_glyph(source, glyph)

show(plot)


# trace = go.Bar(
#     x = ct["cluster_x"],
#     y = ct["NoP_per_cluster"]
# )

# data = [trace]
# iplot(data)

In [ ]:
# Visualisatie 3: gemiddelde snelheid per cluster

x = ct["cluster_x"]
y = ct["kmu_per_cluster"]

plot = figure(plot_width=500, plot_height=500)
source = ColumnDataSource(dict(x=x,top=y,))

glyph = VBar(x="x", top="top", bottom=0, width=0.5, fill_color="#b3de69")
plot.add_glyph(source, glyph)

show(plot)

# trace = go.Bar(
#     x = ct["cluster_x"],
#     y = ct["kmu_per_cluster"]
# )

# data = [trace]
# py.iplot(data)

In [ ]:
#Visualisatie 1: clusters lan - lon

# output to static HTML file
output_file("line.html")

p = figure(plot_width=500, plot_height=500)
p.circle(x=ct["kmu_per_cluster"], y=ct["NoP_per_cluster"], size=20, color="navy", alpha=0.5)
show(p)

# Create a trace
# trace = go.Scatter(
#     x = ct["kmu_per_cluster"],
#     y = ct["NoP_per_cluster"],
#     mode = "markers"
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# py.iplot(data, filename='basic-scatter')

In [ ]:
ct.to_csv("test_stoplocaties_2.csv")

In [ ]:
# https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/

# X = ct["cluster_x"]
# y = ct["kmu_per_cluster"]

# for eps in np.arange(0, 1, 0.001):
#     model = db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree').fit(np.radians(coords))
#     model.fit(X_train, y_train)
#     score = model.score(X_test, y_test)        
#     if score > best_score:
#         best_score = score
#         best_C = C
#         best_gamma = gamma 
# print('Highest Accuracy Score: ', best_score)   

# Randomized Search for Algorithm Tuning
# import numpy as np
# from scipy.stats import uniform as sp_rand
# from sklearn import datasets
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import RandomizedSearchCV

# # prepare a range of alpha values to test
# alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# # create and fit a ridge regression model, testing each alpha
# model = Ridge()
# grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
# grid.fit(dataset., dataset.target)
# print(grid)
# # summarize the results of the grid search
# print(grid.best_score_)
# print(grid.best_estimator_.alpha)

#  itr = 0, eps = 900, minpts = 5, _step = 100
#  while number of outliers < 10  and itr < 10:
#       eps = eps + _step
#       itr = itr + 1
#       dbscan(eps,minpts)

# for epsilon in np.array([1, 0.01, 0.001, 0.0001, 0]):
#     model =DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree').fit(np.radians(coords))


# x = np.arange(6).reshape(2,3)

# for x in model:
#     model = DBSCAN(eps=1, min_samples=1, algorithm='ball_tree').fit(np.radians(coords))

#     print(x)
# model = DBSCAN(eps=1, min_samples=1, algorithm='ball_tree').fit(np.radians(coords))
# for eps in DBSCAN(model):
#     print (x)

# for x in np.array([1,0.1,0.01,0.001,0.0001,0]):
#     print(x)

# df_loop = df_new
# epsilon_2 = 0.001
# coords_speed_2 = np.asarray(df_loop[['Lat_a', 'Lon_a', "speed_kmu"]])
# coords_2 = preprocessing.scale(coords_speed_2)

# def gemiddelde_kmu_per_eps(mean_kmu):
#     model = DBSCAN(eps=epsilon_2, min_samples=1, algorithm='ball_tree').fit(np.radians(coords_2))
#     cluster_labels_2 = db.labels_
#     num_clusters_2 = len(set(cluster_labels_2))
#     print('Number of clusters: {}'.format(num_clusters_2))
#     df_loop["cluster_2"] = cluster_labels_2
#     kmu_per_cluster_2 = df_loop.groupby('cluster_2')['speed_kmu'].mean().reset_index(name='kmu_per_cluster_2')
#     mean_kmu = kmu_per_cluster_2["kmu_per_cluster_2"].mean()
#     print(mean_kmu)

# # y = gemiddelde snelheid per cluster
# # x = eps

# # gemiddelde_kmu_per_eps(mean_kmu)

# print(model)
# # gemiddelde_kmu_per_eps(mean_kmu)
